# T1055.002 Portable Executable Injection

-----------------------------------------------------------------------

## Technique Description

Adversaries may inject portable executables (PE) into processes in order to evade process-based defenses as well as possibly elevate privileges. PE injection is a method of executing arbitrary code in the address space of a separate live process. 

PE injection is commonly performed by copying code (perhaps without a file on disk) into the virtual address space of the target process before invoking it via a new thread. The write can be performed with native Windows API calls such as <code>VirtualAllocEx</code> and <code>WriteProcessMemory</code>, then invoked with <code>CreateRemoteThread</code> or additional code (ex: shellcode). The displacement of the injected code does introduce the additional requirement for functionality to remap memory references. (Citation: Elastic Process Injection July 2017) 

Running code in the context of another process may allow access to the process's memory, system/network resources, and possibly elevated privileges. Execution via PE injection may also evade detection from security products since the execution is masked under a legitimate process. 

## Technique Detection

Monitoring Windows API calls indicative of the various types of code injection may generate a significant amount of data and may not be directly useful for defense unless collected under specific circumstances for known bad sequences of calls, since benign use of API functions may be common and difficult to distinguish from malicious behavior. Windows API calls such as <code>CreateRemoteThread</code> and those that can be used to modify memory within another process, such as <code>VirtualAllocEx</code>/<code>WriteProcessMemory</code>, may be used for this technique.(Citation: Elastic Process Injection July 2017)

Analyze process behavior to determine if a process is performing actions it usually does not, such as opening network connections, reading files, or other suspicious actions that could relate to post-compromise behavior. 

-----------------------------------------------------------------------

### Tactics:

  * Defense-Evasion

  * Privilege-Escalation

### Platforms:

  * Windows

### Adversary Required Permissions:

  * User

### Defenses Bypassed:

  * Anti-virus

  * Application control

### Data Sources:

  * **Process:** OS API Execution

  * **Process:** Process Access

  * **Process:** Process Modification

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
| Rocke | [Rocke](https://attack.mitre.org/groups/G0106)'s miner, "TermsHost.exe", evaded defenses by injecting itself into Windows processes, including Notepad.exe.(Citation: Talos Rocke August 2018)	| 
| Gorgon Group | [Gorgon Group](https://attack.mitre.org/groups/G0078) malware can download a remote access tool, [ShiftyBug](https://attack.mitre.org/software/S0294), and inject into another process.(Citation: Unit 42 Gorgon Group Aug 2018)| 
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1055/002)

  * [Elastic Process Injection July 2017](https://www.endgame.com/blog/technical-blog/ten-process-injection-techniques-technical-survey-common-and-trending-process), Hosseini, A. (2017, July 18). Ten Process Injection Techniques: A Technical Survey Of Common And Trending Process Injection Techniques. Retrieved December 7, 2017.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 8 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- N/A

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Example APT/Threat actor | 1, 2 | 2, 3, 5|

#### Turla	
- Most Turla in-house malware (Carbon, ComRAT, Epic, Gazer, Mosquito, Powerstallion) use process injection's T1055.002 and T1055.003
- Has also used PowerSploit's Invoke-ReflectivePEInjection.ps1 to reflectively load a PowerShell payload into a random process on the victim system.
- Gazer injects its communication module into an Internet accessible process through which it performs C2.

#### Generic	
- Svchost.exe making network connections on tcp/447 and tcp/449
- Notepad.exe making external network connections
- Mshta.exe calling CreateRemoteThread to inject code
- Rundll32.exe, regasm.exe, regsvr32.exe, regsvcs.exe, svchost.exe, and werfault.exe process executions without command-line options may indicate they are targets of process injection.
- Microsoft processes such as vbc.exe with command lines including /scomma, /shtml, or /stext may indicate the injection of Nirsoft tools for credential access
- Linux processes with memfd: in their path indicate they were spawned from code injected into another process.

## Detection Blindspots

- Process injections are inherently difficult to detect, and the most commonly utilized MITRE tactic due to the defense evasion and stealth provided to the adversary. Therefore, an endpoint solution such as Sysmon should be utilized rather than relying solely on organic logging. Suspicious endpoints should undergo memory analysis for scoping and deeper validation.

## Analytical References

  * [Turla Powershell Usage](https://www.welivesecurity.com/2019/05/29/turla-powershell-usage/)
  * [Sysmon Rules (github)](https://github.com/Neo23x0/sigma/blob/master/rules/)
  * [Ten Processing Injection Techniques - Technical Survey - Common and Trending Processes (elastic)](https://www.elastic.co/blog/ten-process-injection-techniques-technical-survey-common-and-trending-process)
  * [Chap System Auditing (redhat)](https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/6/html/security_guide/chap-system_auditing)
  * [Process Injection (redcanary)](https://redcanary.com/threat-detection-report/techniques/process-injection/)
  * [ESET Gazer (welivesecurity)](https://www.welivesecurity.com/wp-content/uploads/2017/08/eset-gazer.pdf)
  * [Introducing Whitebear (securelist)](https://securelist.com/introducing-whitebear/81638/)
  * [Prebuilt Rules (elastic)](https://www.elastic.co/guide/en/siem/guide/current/prebuilt-rules.html)

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Turla is the only Russian APT group that has been witnessed utilizing process injection in the wild thus far (Q3 2020).
- Look for the basics, such as a process with an unusual/non-standard parent or child
- SIGMA rules can be converted into many different SIEM rules/searches/queries. See https://github.com/Neo23x0/sigma.
- Most Auditbeat configurations are insufficient to properly detect linux process injection.
- The Elastalert queries above provide a framework for Kibana queries.
- Memory analysis is the most reliable means to detect process injection.

#### Analytic 1

  * **Information:** Generic CobaltStrike injection

  * **Source:** Memory, Sysmon

  * **Tool:** SIGMA sysmon rule, Kibana, Volatility

  * **Notes:** 

  * **Query Kibana:** ```(EventID:"8" AND TargetProcessAddress.keyword:(*0B80 OR *0C7C OR *0C88))```
  * **Volatility Modules:** `malfind, ldrmodules, and hollowfind modules`

#### Analytic 2

  * **Information:** Monitor for named pipe creation for possible indicators of infected processes with external modules 

  * **Source:** Sysmon, Memory
  
  * **Tool:** SIGMA sysmon rule, Kibana, Volatility

  * **Notes:** 

  * **Query Kibana:** ```(EventID:("17" OR "18") AND PipeName:("\\atctl" OR "\\userpipe" OR "\\iehelper" OR "\\sdlrpc" OR "\\comnap"))```
  
  * **Volatility Modules:** `malfind, ldrmodules, and hollowfind modules`



#### Analytic 3

  * **Information:** Detect Windows API calls that can be used to modify memory within another process

  * **Source:** Memory, Sysmon

  * **Tool:** SIGMA sysmon rule, Kibana, Volatility

  * **Notes:** Monitor DLL/PE file events, specifically creation of these binary files as well as the loading of DLLs into processes. Look for DLLs that are not recognized or not normally loaded into a process.

  * **Query Kibana:** `process.name:MSBuild.exe and event.action:"CreateRemoteThread detected`
  * **Elastalert/Kibana:** `(EventID:"8" AND StartModule.keyword:*\\kernel32.dll AND StartFunction:"LoadLibraryA")`
  * **Volatility:**  `malfind, ldrmodules, dlllist, and hollowfind modules`


#### Analytic 4

  * **Information:** Connection events for possible indicators of infected processes with external modules

  * **Source:** Sysmon

  * **Tool:** SIGMA sysmon rule, Kibana

  * **Notes:** 

  * **Query Kibana:** `process.name:rundll32.exe and event.action:"Network connection detected (rule: NetworkConnect)" and not destination.ip:(10.0.0.0/8 or 172.16.0.0/12 or 192.168.0.0/16 or 127.0.0.0/8)`
  * **Elastalert/Kibana:** `(EventID:("17" OR "18") AND PipeName.keyword:(\\isapi_http OR \\isapi_dg OR \\isapi_dg2 OR \\sdlrpc OR \\ahexec OR \\winsession OR \\lsassw OR \\46a676ab7f179e511e30dd2dc41bd388 OR \\9f81f59bc58452127884ce513865ed20 OR \\e710f28d59aa529d6792ca6ff0ca1b34 OR \\rpchlp_3 OR \\NamePipe_MoreWindows OR \\pcheap_reuse OR \\msagent_* OR \\gruntsvc))`

#### Analytic 5

  * **Information:** Rundll32.exe, regasm.exe, regsvr32.exe, regsvcs.exe, svchost.exe, and werfault.exe process executions without command-line options may indicate they are targets of process injection.

  * **Source:** Memory, Sysmon

  * **Tool:** SIGMA sysmon rule, Kibana, Volatility

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query Kibana:** ```process.name:(RegAsm.exe or RegSvcs.exe) and event.action:"Process```
  * **Elastalert/Kibana:** ```Create (rule: ProcessCreate)" /// (EventID:"8" AND SourceImage.keyword:*\\powershell.exe AND TargetImage.keyword:*\\rundll32.exe) ```    

  * **Volatility:**  malfind, ldrmodules, and hollowfind modules

#### Analytic 6

  * **Information:** Linux system call monitoring

  * **Source:** Memory, AuditBeat

  * **Tool:** Kibana, Volatility

  * **Notes:** Monitoring for Linux specific calls such as the ptrace system call should not generate large amounts of data due to their specialized nature, and can be a very effective method to detect some of the common process injection methods. On Linux systems, modifying the /etc/ld.so.preload file or the environment variables LD_PRELOAD or LD_LIBRARY_PATH may allow injection.

  * **Query Kibana Pseudo:** `${LD_PRELOAD} OR ${LD_LIBRARY_PATH}`
  * **Volatility:** `linux_malfind, linux_ldrmodules, linux_process_hollow`


-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

#### Analytic 2

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

